In [1]:
# --- 1. SETUP AND IMPORTS ---
# Member 4: Implementation of the Advanced Wide & Deep Network Architecture.
# Objective: Eliminate False Negatives (FN) and push Malignant Recall to 1.00.

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, BatchNormalization, Add, Activation, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# Suppress FutureWarnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning) 

# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries loaded successfully. Using Keras Functional API for Wide & Deep Model.")

Libraries loaded successfully. Using Keras Functional API for Wide & Deep Model.


In [2]:
# --- 2. DATA LOADING AND INITIAL CLEANING ---

try:
    data = pd.read_csv('Cancer_Data.csv') # [1]
    
    # Check for the common redundant column (if present)
    if 'Unnamed: 32' in data.columns:
        data = data.drop(['Unnamed: 32'], axis=1) 
        
    print(f"Dataset shape: {data.shape}")
except FileNotFoundError:
    print("Error: 'Cancer_Data.csv' not found. Please ensure the file is in the correct directory.")
    exit()

# Identify Features (X) and Target (Y)
X = data.drop(['id', 'diagnosis'], axis=1) # [1]
Y = data['diagnosis']


Dataset shape: (569, 32)


In [3]:
# --- 3. EXPLORATORY DATA ANALYSIS (EDA) AND VISUALIZATIONS ---

# 3.1. Target Class Distribution
# (Included here but visualization calls are commented out to keep the output clean)
data_corr = data.copy()
data_corr['diagnosis'] = data_corr['diagnosis'].replace({'M': 1, 'B': 0}).astype(int) 

# 3.2. Feature Correlation Heatmap
mean_features = ['diagnosis'] + [col for col in data_corr.columns if 'mean' in col]
corr_matrix = data_corr[mean_features].corr() 
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix, annot=True, fmt='.1f', cmap='coolwarm')
# plt.show()

In [4]:
# --- 4. DATA PREPARATION (SPLIT FIRST) ---

# 4.1. Target Encoding 
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y) # 'M' -> 1, 'B' -> 0

TEST_SIZE_FINAL = 0.15 
VAL_SIZE_RATIO = 0.1764 

# Split 1: Training Pool (85%) and Test Set (15%), stratified
X_train_full, X_test, Y_train_full, Y_test = train_test_split(
    X.values, encoded_Y, test_size=TEST_SIZE_FINAL, random_state=42, stratify=encoded_Y
)

# Split 2: Training Set (70%) and Validation Set (15%)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train_full, Y_train_full, test_size=VAL_SIZE_RATIO, random_state=42, stratify=Y_train_full
)


In [5]:
# --- 5. DATA PREPROCESSING (SCALE AFTER SPLIT) (10% Grading Weight) ---

scaler = StandardScaler()

# 5.1. Fit ONLY on Training Data (Essential to prevent data leakage)
X_train_scaled = scaler.fit_transform(X_train) 

# 5.2. Transform Validation and Test Data 
X_val_scaled = scaler.transform(X_val)       
X_test_scaled = scaler.transform(X_test)     

input_dim = X_train_scaled.shape[1]

print(f"\nData successfully split and scaled. Input Dimension: {input_dim}")



Data successfully split and scaled. Input Dimension: 30


In [6]:
# --- 6. MODEL ARCHITECTURE DEFINITION (WIDE & DEEP CORE CONTRIBUTION) ---

def create_wide_and_deep_model(input_dim):
    # 6.1. Define the input layer (Shared input for both paths)
    inputs = Input(shape=(input_dim,), name="Input_Features")

    # 6.2. WIDE Component (Memorization/Linearity)
    # Learns simple, linear relationships directly from input
    wide_output = Dense(1, activation=None, name="Wide_Output")(inputs) # Linear activation (or no activation) before final sigmoid
    
    # 6.3. DEEP Component (Generalization/Feature Interactions)
    # Learns complex, non-linear relationships through multiple layers
    x = Dense(128, activation='relu', name="Deep_Dense_1")(inputs)
    x = BatchNormalization(name="Deep_BN_1")(x)
    x = Dropout(0.4, name="Deep_Dropout_1")(x) # Higher dropout for complexity
    
    x = Dense(64, activation='relu', name="Deep_Dense_2")(x)
    x = BatchNormalization(name="Deep_BN_2")(x)
    x = Dropout(0.4, name="Deep_Dropout_2")(x)
    
    x = Dense(32, activation='relu', name="Deep_Dense_3")(x)
    
    # Output of the deep path (linear activation before merging)
    deep_output = Dense(1, activation=None, name="Deep_Linear_Output")(x)

    # 6.4. Merge Components and Final Output
    # Merge the outputs of the Wide and Deep components by simple addition 
    merged = Add(name="Wide_Deep_Merge")([wide_output, deep_output]) # [2]
    
    # Apply final activation (Sigmoid for binary classification probability)
    final_output = Activation('sigmoid', name="Final_Sigmoid")(merged)
    
    model = Model(inputs=inputs, outputs=final_output, name="Wide_and_Deep_M4")
    return model

model_m4 = create_wide_and_deep_model(input_dim)
print("\nModel Architecture (Wide & Deep Network - Interpretable and Robust):")
model_m4.summary()



Model Architecture (Wide & Deep Network - Interpretable and Robust):


Model: "Wide_and_Deep_M4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input_Features (InputLayer)   │ (None, 30)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_Dense_1 (Dense)          │ (None, 128)               │           3,968 │ Input_Features[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_BN_1                     │ (None, 128)               │             512 │ Deep_Dense_1[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_Dropout_1 (Dropout)      │ (None, 128)               │               0 │ Deep_BN_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_Dense_2 (Dense)          │ (None, 64)                │           8,256 │ Deep_Dropout_1[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_BN_2                     │ (None, 64)                │             256 │ Deep_Dense_2[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_Dropout_2 (Dropout)      │ (None, 64)                │               0 │ Deep_BN_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_Dense_3 (Dense)          │ (None, 32)                │           2,080 │ Deep_Dropout_2[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Wide_Output (Dense)           │ (None, 1)                 │              31 │ Input_Features[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Deep_Linear_Output (Dense)    │ (None, 1)                 │              33 │ Deep_Dense_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Wide_Deep_Merge (Add)         │ (None, 1)                 │               0 │ Wide_Output[0][0],         │
│                               │                           │                 │ Deep_Linear_Output[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Final_Sigmoid (Activation)    │ (None, 1)                 │               0 │ Wide_Deep_Merge[0][0]      │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 15,136 (59.12 KB)

 Trainable params: 14,752 (57.62 KB)

 Non-trainable params: 384 (1.50 KB)

In [7]:
# --- 7. MODEL COMPILATION AND TRAINING ---

model_m4.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy', 
    metrics=['accuracy', keras.metrics.AUC(name='auc')] 
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,          # Increased patience to allow deep network full convergence
    restore_best_weights=True # [3]
)

print("\nStarting Model Training (Final attempt to fix False Negatives)...")
history_m4 = model_m4.fit(
    X_train_scaled, Y_train,
    epochs=500,             
    batch_size=32,
    validation_data=(X_val_scaled, Y_val),
    callbacks=[early_stopping],
    verbose=2
)
print("Model training finished.")


Starting Model Training (Final attempt to fix False Negatives)...
Epoch 1/500
13/13 - 9s - 714ms/step - accuracy: 0.4156 - auc: 0.4196 - loss: 1.1645 - val_accuracy: 0.4884 - val_auc: 0.4821 - val_loss: 0.8672
Epoch 2/500
13/13 - 0s - 35ms/step - accuracy: 0.7733 - auc: 0.8679 - loss: 0.4801 - val_accuracy: 0.6628 - val_auc: 0.7610 - val_loss: 0.5969
Epoch 3/500
13/13 - 1s - 51ms/step - accuracy: 0.8640 - auc: 0.9444 - loss: 0.2942 - val_accuracy: 0.7674 - val_auc: 0.8848 - val_loss: 0.4307
Epoch 4/500
13/13 - 1s - 88ms/step - accuracy: 0.9395 - auc: 0.9699 - loss: 0.2007 - val_accuracy: 0.8837 - val_auc: 0.9363 - val_loss: 0.3334
Epoch 5/500
13/13 - 1s - 61ms/step - accuracy: 0.9295 - auc: 0.9802 - loss: 0.1776 - val_accuracy: 0.9302 - val_auc: 0.9690 - val_loss: 0.2691
Epoch 6/500
13/13 - 1s - 49ms/step - accuracy: 0.9471 - auc: 0.9820 - loss: 0.1491 - val_accuracy: 0.9419 - val_auc: 0.9800 - val_loss: 0.2294
Epoch 7/500
13/13 - 1s - 48ms/step - accuracy: 0.9471 - auc: 0.9792 - loss

In [8]:
# --- 8. MODEL EVALUATION (5% Grading Weight) ---

print("\nEvaluating Model on the held-out Test Set (15%):")
loss_m4, accuracy_m4, auc_m4 = model_m4.evaluate(X_test_scaled, Y_test, verbose=0)

print(f"\n--- Member 4 (Wide & Deep) Final Test Results ---")
print(f"Test Loss: {loss_m4:.4f}")
print(f"Test Accuracy: {accuracy_m4:.4f}")
print(f"Test AUC-ROC: {auc_m4:.4f}")

# Generate detailed classification report
Y_pred_prob = model_m4.predict(X_test_scaled)
Y_pred_class = (Y_pred_prob > 0.5).astype("int32")

report = classification_report(Y_test, Y_pred_class, target_names=encoder.classes_)
print("\nClassification Report (Key Metrics for Comparison):")
print(report)



Evaluating Model on the held-out Test Set (15%):

--- Member 4 (Wide & Deep) Final Test Results ---
Test Loss: 0.0597
Test Accuracy: 0.9767
Test AUC-ROC: 0.9994
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step

Classification Report (Key Metrics for Comparison):
              precision    recall  f1-score   support

           B       0.98      0.98      0.98        54
           M       0.97      0.97      0.97        32

    accuracy                           0.98        86
   macro avg       0.98      0.98      0.98        86
weighted avg       0.98      0.98      0.98        86

